# Dataset Generation Pipeline

This notebook processes and cleans multiple data sources to create a unified dataset for analysis. The pipeline handles:

1. **Water consumption data** (`consum_telelectura_v2.parquet`) - Daily water consumption readings
2. **Leak incident data** (`consum_avisos_missatges_v2.parquet`) - Water leak reports and messages
3. **Weather data** (`Dades_meteorològiques_diàries_de_la_XEMA`) - Daily meteorological measurements
4. **Socioeconomic index** (`ist_per_seccio_censal.csv`) - Territorial socioeconomic index by census section

All data is filtered to Barcelona (census sections starting with 8019/08019) and processed in chunks to handle large file sizes efficiently.


## 1. Setup

Import required libraries and establish database connection. We use DuckDB for efficient data loading and pandas for data manipulation.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
import duckdb
con = duckdb.connect()

## 2. Data Loading

Load and inspect all raw data sources. We use DuckDB to efficiently read large parquet files without loading everything into memory at once. This allows us to check data sizes and sample records before processing.


### 2.1 Water Consumption Data

The consumption dataset contains daily water meter readings. We check the total number of records and inspect a sample to understand the data structure.


In [3]:
# Check consumption data size and sample
# We'll load data only when needed for pandas operations
print("Checking consum data...")
row_count = con.execute("SELECT COUNT(*) as count FROM 'data/consum_telelectura_v2.parquet'").df()['count'].iloc[0]
print(f"Total rows in consum: {row_count:,}")

# Load a sample for inspection
df_consum_sample = con.execute("SELECT * FROM 'data/consum_telelectura_v2.parquet' LIMIT 5").df()
print("\nSample data:")
df_consum_sample.head()

Checking consum data...
Total rows in consum: 17,112,709

Sample data:


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025,C,0,3,D15TD031058M,2016-04-25,ITR,23,30
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025,C,0,3,D15TD031058M,2016-04-25,ITR,23,30
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025,C,0,3,D15TD031058M,2016-04-25,ITR,23,30
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025,C,0,3,D15TD031058M,2016-04-25,ITR,23,30
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025,C,0,3,D15TD031058M,2016-04-25,ITR,23,30


### 2.2 Leak Incidents Data

The leak incidents dataset contains water leak reports and messages. This is a very large dataset (over 76 million rows), so we'll process it in chunks.


In [4]:
# Check leak incidents data size and sample
print("Checking fuites data...")
row_count = con.execute("SELECT COUNT(*) as count FROM 'data/consum_avisos_missatges_v2.parquet'").df()['count'].iloc[0]
print(f"Total rows in fuites: {row_count:,}")

# Load a sample for inspection
df_fuites_sample = con.execute("SELECT * FROM 'data/consum_avisos_missatges_v2.parquet' LIMIT 5").df()
print("\nSample data:")
df_fuites_sample.head()

Checking fuites data...
Total rows in fuites: 76,372,248

Sample data:


,POLISSA_SUBM,DATA_INI_FACT,DATA_FIN_FACT,CREATED_MENSAJE,CODIGO_MENSAJE,TIPO_MENSAJE,US_AIGUA_SUBM,SECCIO_CENSAL,NUMEROSERIECONTADOR,CONSUMO_REAL,FECHA_HORA
0,RGYFWIZ4ZRRZKX2K,2023-09-13 00:00:00,2023-11-14 00:00:00,NaT,None,None,DOMÈSTIC,0801907090,IBAJ44VHSIRRTASA,NaN,2024-01-01
1,HHB4U5HUQKW7IOGD,2023-08-13 00:00:00,2023-10-16 00:00:00,NaT,None,None,DOMÈSTIC,0801909040,L2CLPPJRIPAEESV7,NaN,2024-01-01
2,EU6AT3IKPUKCZTBU,2024-01-24 00:00:00,2024-03-26 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,45TBDJQN4LA37ZIN,NaN,2024-01-01
3,EU6AT3IKPUKCZTBU,2023-11-27 00:00:00,2024-01-24 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,45TBDJQN4LA37ZIN,NaN,2024-01-01
4,EWNDTPECBVEGW6AU,2023-09-29 00:00:00,2023-11-27 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,VTRAI3L24SWKVC5H,NaN,2024-01-01


### 2.3 Socioeconomic Index Data

The socioeconomic index provides territorial indicators at the census section level, which will be used for joining with consumption and leak data.


In [5]:
# Dataset Índex Socioeconòmic Territorial per secció censal
print("Loading ist_per_seccio_censal.csv with DuckDB...")
df_socio = con.execute("""
    SELECT * FROM read_csv_auto('data/ist_per_seccio_censal.csv', 
                                 delim=';', 
                                 decimal_separator=',',
                                 header=true)
""").df()
print(f"Loaded {len(df_socio):,} rows")
df_socio.head()

Loading ist_per_seccio_censal.csv with DuckDB...
Loaded 5,092 rows


,any,secció censal,concepte,estat,valor
0,2022,25001901001,Índex socioeconòmic territorial,None,97.8
1,2022,08001801001,Índex socioeconòmic territorial,None,105.1
2,2022,08001801002,Índex socioeconòmic territorial,None,102.9
3,2022,08001801003,Índex socioeconòmic territorial,None,118.5
4,2022,08001801004,Índex socioeconòmic territorial,None,118.0


### 2.4 Weather Data

Weather data from XEMA (Catalan Meteorological Service) contains daily measurements from multiple weather stations. This data will be used to analyze correlations between weather patterns and water consumption/leaks.


In [6]:
# Load weather data using DuckDB
print("Loading dades meteorloògiques diàries with DuckDB...")
df_weather = con.execute("SELECT * FROM read_csv_auto('data/Dades_meteorològiques_diàries_de_la_XEMA_20251119.csv', header=true)").df()
print(f"Loaded {len(df_weather):,} rows")

df_weather.head()

Loading dades meteorloògiques diàries with DuckDB...
Loaded 129,301 rows


,ID,CODI_ESTACIO,NOM_ESTACIO,DATA_LECTURA,CODI_VARIABLE,NOM_VARIABLE,VALOR,UNITAT,HORA _TU,Estat
0,D51000012101,D5,Barcelona - Observatori Fabra,2021-01-01,1.000,Temperatura mitjana diària,"5,3",°C,<NA>,Representatiu
1,D51001012101,D5,Barcelona - Observatori Fabra,2021-01-01,1.001,Temperatura màxima diària + hora,"6,5",°C,13:06:00,Representatiu
2,D51002012101,D5,Barcelona - Observatori Fabra,2021-01-01,1.002,Temperatura mínima diària + hora,"3,1",°C,23:56:00,Representatiu
3,D51003012101,D5,Barcelona - Observatori Fabra,2021-01-01,1.003,Temperatura mitjana diària clàssica,"4,8",°C,<NA>,Representatiu
4,D51004012101,D5,Barcelona - Observatori Fabra,2021-01-01,1.004,Amplitud tèrmica diària,"3,4",°C,<NA>,Representatiu


## 3. Data Cleaning and Filtering

Process the raw data files to extract Barcelona-specific data and clean weather data. Since the datasets are very large, we process them in chunks (split files) to avoid memory issues. For consumption and leak data, we:

1. Filter to Barcelona census sections (8019/08019)
2. Transform census section codes to the standard 11-digit format
3. Keep only relevant columns
4. Save cleaned data:
   - Consumption: Split parquet files in `clean/split_consum_bcn/` (kept as splits due to size)
   - Leaks: Single parquet file `clean/fuites_clean_bcn.parquet` (one leak per row)

For weather data, we clean missing values and ensure consistent formatting.


### 3.1 Clean Consumption Data

Process consumption data split files:
- Filter to Barcelona (SECCIO_CENSAL starting with 8019)
- Transform census section codes: add '3' after 8019, then prepend '0' to get 11-digit format
- Keep relevant columns: POLIZA_SUMINISTRO, FECHA, CONSUMO_REAL, SECCIO_CENSAL, US_AIGUA_GEST, DATA_INST_COMP
- Convert date columns to datetime format
- Save cleaned Barcelona data as split parquet files: `clean/split_consum_bcn/consum_clean_bcn_part_*.parquet`


In [7]:
# df_consum
# Clean and filter df_consum data from split files

import glob
from pathlib import Path

# Locate (or create) the directory that should contain the split chunks
split_dir = Path("data/split_consum")
split_dir.mkdir(parents=True, exist_ok=True)
split_pattern = "consum_telelectura_v2_part_*.parquet"
split_files = sorted(glob.glob(str(split_dir / split_pattern)))

if not split_files:
    raw_consum_file = Path("data/consum_telelectura_v2.parquet")
    if raw_consum_file.exists():
        print("No pre-split consumption files found. Splitting the raw file now (one-time operation)...")
        from split_parquet import split_parquet_efficient

        split_chunk_size = 1_000_000
        split_parquet_efficient(str(raw_consum_file), str(split_dir), chunk_size=split_chunk_size)
        split_files = sorted(glob.glob(str(split_dir / split_pattern)))
    else:
        print("⚠ No split files found and raw file 'data/consum_telelectura_v2.parquet' is missing.")
        split_files = []

if not split_files:
    raise FileNotFoundError("Unable to locate consumption split files even after attempting to split the raw file.")

print(f"Found {len(split_files)} split files to process")
print("-" * 80)

# Create output directory for cleaned files
output_dir = Path("clean/split_consum_bcn")
output_dir.mkdir(parents=True, exist_ok=True)

# Keep only relevant columns
cols_keep_consum = [
    'POLIZA_SUMINISTRO', 'FECHA', 'CONSUMO_REAL', 'SECCIO_CENSAL', 'US_AIGUA_GEST', 'DATA_INST_COMP'
]

# Track statistics
total_rows_processed = 0
total_rows_bcn = 0

# Process each split file
for i, file_path in enumerate(split_files, 1):
    print(f"\nProcessing file {i}/{len(split_files)}: {Path(file_path).name}")
    
    # Load one split file
    df_chunk = pd.read_parquet(file_path)
    total_rows_processed += len(df_chunk)
    
    # Convert FECHA and DATA_INST_COMP to datetime
    for date_col in ["FECHA", "DATA_INST_COMP"]:
        if date_col in df_chunk.columns:
            df_chunk[date_col] = pd.to_datetime(df_chunk[date_col], errors="coerce")
    
    # Keep only relevant columns
    df_chunk_relevant = df_chunk[cols_keep_consum].copy()
    
    # Create df_consum_bcn: First filter to keep only rows where SECCIO_CENSAL starts with 8019
    # Convert SECCIO_CENSAL to string to check if it starts with 8019
    df_chunk_relevant['SECCIO_CENSAL'] = df_chunk_relevant['SECCIO_CENSAL'].astype(str)
    mask_bcn = df_chunk_relevant['SECCIO_CENSAL'].str.startswith('8019', na=False)
    df_chunk_bcn = df_chunk_relevant[mask_bcn].copy()
    
    # Then transform SECCIO_CENSAL: add 3 after 8019, then add 0 before all digits to get 11-digit strings
    df_chunk_bcn['SECCIO_CENSAL'] = df_chunk_bcn['SECCIO_CENSAL'].str.replace('^8019', '80193', regex=True)
    df_chunk_bcn['SECCIO_CENSAL'] = '0' + df_chunk_bcn['SECCIO_CENSAL']
    
    # Save cleaned split file
    output_file = output_dir / f"consum_clean_bcn_part_{i:03d}.parquet"
    df_chunk_bcn.to_parquet(output_file, index=False)
    
    total_rows_bcn += len(df_chunk_bcn)
    print(f"Processed {len(df_chunk):,} rows → {len(df_chunk_bcn):,} Barcelona rows")
    print(f"Saved to {output_file.name}")

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total rows processed: {total_rows_processed:,}")
print(f"Total Barcelona rows (after filtering): {total_rows_bcn:,}")
print(f"Cleaned files saved to: {output_dir}/")
print(f"\nFirst rows sample from last processed file:")
print(df_chunk_bcn.head())

No pre-split consumption files found. Splitting the raw file now (one-time operation)...
Reading parquet file: data/consum_telelectura_v2.parquet
Chunk size: 1,000,000 rows
Output directory: data/split_consum
------------------------------------------------------------
Total rows in file: 17,112,709
Number of row groups: 17
Will create approximately 18 output files
------------------------------------------------------------
Reading row group 1/17... loaded 1,048,576 rows
  ✓ Chunk 1: Saved 1,000,000 rows to consum_telelectura_v2_part_001.parquet
Reading row group 2/17... loaded 1,048,576 rows
  ✓ Chunk 2: Saved 1,000,000 rows to consum_telelectura_v2_part_002.parquet
Reading row group 3/17... loaded 1,048,576 rows
  ✓ Chunk 3: Saved 1,000,000 rows to consum_telelectura_v2_part_003.parquet
Reading row group 4/17... loaded 1,048,576 rows
  ✓ Chunk 4: Saved 1,000,000 rows to consum_telelectura_v2_part_004.parquet
Reading row group 5/17... loaded 1,048,576 rows
  ✓ Chunk 5: Saved 1,000,00

### 3.2 Clean Leak Incidents Data

Process leak incidents data split files:
- Drop rows where both CODIGO_MENSAJE and CREATED_MENSAJE are missing (invalid records)
- Filter to keep only CODIGO_MENSAJE = "FUITA" (remove "REITERACIÓ DE FUITA" and others)
- Convert CREATED_MENSAJE to date only (year-month-day, no time component)
- Remove duplicates based on POLISSA_SUBM and CREATED_MENSAJE (ensures one leak per row)
- Filter to Barcelona (SECCIO_CENSAL starting with 08019)
- Transform census section codes: add '3' after 08019 to get standard format
- Keep relevant columns: POLISSA_SUBM, CREATED_MENSAJE, CODIGO_MENSAJE, US_AIGUA_SUBM, SECCIO_CENSAL
- Save cleaned Barcelona data as a single parquet file: `clean/fuites_clean_bcn.parquet`

**Result**: One leak per row (one leak per POLISSA_SUBM per day)


In [8]:
# df_fuites - Process split files one at a time to avoid memory issues
# Clean and filter df_fuites data from split files

import glob
from pathlib import Path

# Locate (or create) the directory with the avisos split files
split_dir = Path("data/split_avisos")
split_dir.mkdir(parents=True, exist_ok=True)
split_pattern = "consum_avisos_missatges_v2_part_*.parquet"
split_files = sorted(glob.glob(str(split_dir / split_pattern)))

if not split_files:
    raw_avisos_file = Path("data/consum_avisos_missatges_v2.parquet")
    if raw_avisos_file.exists():
        print("No pre-split leak files found. Splitting the raw avisos file now (one-time operation)...")
        from split_parquet import split_parquet_efficient

        split_chunk_size = 1_000_000
        split_parquet_efficient(str(raw_avisos_file), str(split_dir), chunk_size=split_chunk_size)
        split_files = sorted(glob.glob(str(split_dir / split_pattern)))
    else:
        print("⚠ No split files found and raw file 'data/consum_avisos_missatges_v2.parquet' is missing.")
        split_files = []

if not split_files:
    raise FileNotFoundError("Unable to locate leak split files even after attempting to split the raw file.")

print(f"Found {len(split_files)} split files to process")
print("=" * 80)

# Create output directory for cleaned files
output_dir = Path("clean")
output_dir.mkdir(parents=True, exist_ok=True)

# Keep only relevant columns
cols_keep_fuites = [
    'POLISSA_SUBM', 'CREATED_MENSAJE', 'CODIGO_MENSAJE',
    'US_AIGUA_SUBM', 'SECCIO_CENSAL'
]

# Track statistics
total_rows_processed = 0
total_rows_dropped_both_missing = 0
total_rows_dropped_fuita = 0
total_rows_dropped_duplicates = 0
total_rows_bcn = 0

# List to collect all cleaned data
fuites_clean_list = []

# Process each split file
for i, file_path in enumerate(split_files, 1):
    print(f"\nProcessing file {i}/{len(split_files)}: {Path(file_path).name}")
    
    # Load one split file
    df_chunk = pd.read_parquet(file_path)
    total_rows_processed += len(df_chunk)
    
    # Keep only relevant columns
    df_chunk_relevant = df_chunk[cols_keep_fuites].copy()
    
    # 1. Drop rows where both CODIGO_MENSAJE and CREATED_MENSAJE are missing
    mask_both_missing = df_chunk_relevant['CODIGO_MENSAJE'].isna() & df_chunk_relevant['CREATED_MENSAJE'].isna()
    rows_dropped = mask_both_missing.sum()
    total_rows_dropped_both_missing += rows_dropped
    df_chunk_relevant = df_chunk_relevant[~mask_both_missing].copy()
    
    # 2. Filter to keep only CODIGO_MENSAJE = "FUITA" (remove "REITERACIÓ DE FUITA" and others)
    mask_fuita = df_chunk_relevant['CODIGO_MENSAJE'] == 'FUITA'
    rows_before_fuita_filter = len(df_chunk_relevant)
    df_chunk_relevant = df_chunk_relevant[mask_fuita].copy()
    rows_dropped_fuita = rows_before_fuita_filter - len(df_chunk_relevant)
    
    # 3. Convert CREATED_MENSAJE to datetime and extract date only (year-month-day, no time)
    df_chunk_relevant['CREATED_MENSAJE'] = pd.to_datetime(df_chunk_relevant['CREATED_MENSAJE'], errors='coerce')
    # Drop rows where CREATED_MENSAJE is NaT (invalid dates)
    rows_before_date_filter = len(df_chunk_relevant)
    df_chunk_relevant = df_chunk_relevant[df_chunk_relevant['CREATED_MENSAJE'].notna()].copy()
    rows_dropped_invalid_date = rows_before_date_filter - len(df_chunk_relevant)
    # Extract date only (year-month-day, no time component)
    df_chunk_relevant['CREATED_MENSAJE'] = df_chunk_relevant['CREATED_MENSAJE'].dt.date
    
    # 4. Remove duplicates based on POLISSA_SUBM and CREATED_MENSAJE (date only)
    # This ensures one leak per row (one leak per POLISSA_SUBM per day)
    rows_before_dedup = len(df_chunk_relevant)
    df_chunk_relevant = df_chunk_relevant.drop_duplicates(subset=['POLISSA_SUBM', 'CREATED_MENSAJE'], keep='first')
    rows_dropped_duplicates = rows_before_dedup - len(df_chunk_relevant)
    
    # 5. Filter to keep only SECCIO_CENSAL starting with '08019' which corresponds to Barcelona
    df_chunk_relevant['SECCIO_CENSAL'] = df_chunk_relevant['SECCIO_CENSAL'].astype(str)
    mask_bcn_fuites = df_chunk_relevant['SECCIO_CENSAL'].str.startswith('08019', na=False)
    df_chunk_bcn = df_chunk_relevant[mask_bcn_fuites].copy()
    
    # 6. Transform SECCIO_CENSAL: add 3 after 08019 (no need to add 0, it's already included)
    df_chunk_bcn['SECCIO_CENSAL'] = df_chunk_bcn['SECCIO_CENSAL'].str.replace('^08019', '080193', regex=True)
    
    # Collect cleaned data
    fuites_clean_list.append(df_chunk_bcn)
    total_rows_bcn += len(df_chunk_bcn)
    total_rows_dropped_fuita += rows_dropped_fuita
    total_rows_dropped_duplicates += rows_dropped_duplicates
    print(f"  ✓ Processed {len(df_chunk):,} rows")
    print(f"  ✓ Dropped {rows_dropped:,} rows (both CODIGO_MENSAJE and CREATED_MENSAJE missing)")
    print(f"  ✓ Dropped {rows_dropped_fuita:,} rows (CODIGO_MENSAJE != 'FUITA')")
    print(f"  ✓ Dropped {rows_dropped_invalid_date:,} rows (invalid CREATED_MENSAJE date)")
    print(f"  ✓ Dropped {rows_dropped_duplicates:,} duplicate rows (same POLISSA_SUBM + date)")
    print(f"  ✓ After filtering Barcelona: {len(df_chunk_bcn):,} rows (one leak per row)")

# Combine all cleaned data
print("\n" + "=" * 80)
print("COMBINING ALL CLEANED DATA")
print("=" * 80)
print("Combining all cleaned data...")
df_fuites_clean = pd.concat(fuites_clean_list, ignore_index=True)

# Remove any duplicates that might exist across files (same POLISSA_SUBM + CREATED_MENSAJE)
rows_before_final_dedup = len(df_fuites_clean)
df_fuites_clean = df_fuites_clean.drop_duplicates(subset=['POLISSA_SUBM', 'CREATED_MENSAJE'], keep='first')
rows_dropped_final_duplicates = rows_before_final_dedup - len(df_fuites_clean)

# Save as single parquet file
output_file = output_dir / "fuites_clean_bcn.parquet"
print(f"\nSaving to {output_file}...")
df_fuites_clean.to_parquet(output_file, index=False)

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total rows processed: {total_rows_processed:,}")
print(f"Total rows dropped (both CODIGO_MENSAJE and CREATED_MENSAJE missing): {total_rows_dropped_both_missing:,}")
print(f"Total rows dropped (CODIGO_MENSAJE != 'FUITA'): {total_rows_dropped_fuita:,}")
print(f"Total rows dropped (duplicates: same POLISSA_SUBM + date): {total_rows_dropped_duplicates:,}")
print(f"Total rows dropped (final deduplication across files): {rows_dropped_final_duplicates:,}")
print(f"Total Barcelona rows (after all filtering, one leak per row): {len(df_fuites_clean):,}")
print(f"Cleaned file saved to: {output_file}")
print(f"\nFirst rows sample:")
print(df_fuites_clean.head())
print(f"\nDataset shape: {df_fuites_clean.shape}")
print(f"Columns: {list(df_fuites_clean.columns)}")

No pre-split leak files found. Splitting the raw avisos file now (one-time operation)...
Reading parquet file: data/consum_avisos_missatges_v2.parquet
Chunk size: 1,000,000 rows
Output directory: data/split_avisos
------------------------------------------------------------
Total rows in file: 76,372,248
Number of row groups: 73
Will create approximately 77 output files
------------------------------------------------------------
Reading row group 1/73... loaded 1,048,576 rows
  ✓ Chunk 1: Saved 1,000,000 rows to consum_avisos_missatges_v2_part_001.parquet
Reading row group 2/73... loaded 1,048,576 rows
  ✓ Chunk 2: Saved 1,000,000 rows to consum_avisos_missatges_v2_part_002.parquet
Reading row group 3/73... loaded 1,048,576 rows
  ✓ Chunk 3: Saved 1,000,000 rows to consum_avisos_missatges_v2_part_003.parquet
Reading row group 4/73... loaded 1,048,576 rows
  ✓ Chunk 4: Saved 1,000,000 rows to consum_avisos_missatges_v2_part_004.parquet
Reading row group 5/73... loaded 1,048,576 rows
  

### 3.3 Clean Weather Data

Process weather data:
- Replace null values in HORA_TU with "NA" (only extreme values have time information)
- This cleaning step ensures consistent data format for analysis

In [ ]:
from pathlib import Path
from datetime import time


def normalize_hour_column(series, col_name):
    """Convert datetime.time objects to strings and normalize missing values."""
    # Treat blank strings or placeholder values as missing
    series = series.replace(r'^\s*$', pd.NA, regex=True)
    series = series.replace(['NA', 'NaN', 'nan'], pd.NA)
    null_count = series.isna().sum()
    print(f"Detected {null_count:,} missing values in '{col_name}' before fill")

    def convert_time_to_str(val):
        if pd.isna(val):
            return pd.NA
        if isinstance(val, time):
            return val.strftime('%H:%M:%S')
        return str(val)

    series = series.apply(convert_time_to_str)
    series = series.fillna('NA')
    return series, null_count


if 'HORA _TU' in df_weather.columns:
    df_weather['HORA _TU'], null_count = normalize_hour_column(df_weather['HORA _TU'], 'HORA _TU')
    print(f"Replaced missing values in 'HORA _TU' with 'NA' and converted times to strings")
elif 'HORA_TU' in df_weather.columns:
    df_weather['HORA_TU'], null_count = normalize_hour_column(df_weather['HORA_TU'], 'HORA_TU')
    print(f"Replaced missing values in 'HORA_TU' with 'NA' and converted times to strings")
else:
    print("⚠ Column 'HORA _TU' or 'HORA_TU' not found in weather data")

# Convert VALOR column (comma decimal) to numeric
if 'VALOR' in df_weather.columns:
    df_weather['VALOR_NUM'] = (
        df_weather['VALOR']
        .astype(str)
        .str.replace('.', '', regex=False)  # remove thousand separators if any
        .str.replace(',', '.', regex=False)
    )
    df_weather['VALOR_NUM'] = pd.to_numeric(df_weather['VALOR_NUM'], errors='coerce')
else:
    print("⚠ Column 'VALOR' not found in weather data")

print("\nSample of cleaned weather data:")
print(df_weather.head())

# Save cleaned weather data
output_dir = Path("clean")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "weather_clean.parquet"
df_weather.to_parquet(output_file, index=False)
print(f"\n✓ Saved cleaned weather data to {output_file}")

### 3.4 Clean Socioeconomic Data

Process socioeconomic index data:
- Rename column to match other datasets ("secció censal" → "SECCIO_CENSAL")
- Filter to Barcelona (SECCIO_CENSAL starting with "080193")
- Remove "estat" column (100% null values)
- Save cleaned data to clean directory

In [ ]:
from pathlib import Path

# Filter df_socio for Barcelona and remove "estat" column
# First, rename "secció censal" to "SECCIO_CENSAL" to match other datasets
if 'secció censal' in df_socio.columns:
    df_socio = df_socio.rename(columns={'secció censal': 'SECCIO_CENSAL'})

# Filter for Barcelona: SECCIO_CENSAL starting with "080193"
mask_bcn = df_socio['SECCIO_CENSAL'].str.startswith('080193', na=False)
df_socio_bcn = df_socio[mask_bcn].copy()

print(f"Filtering df_socio for Barcelona (SECCIO_CENSAL starting with '080193'):")
print(f"  Original rows: {len(df_socio):,}")
print(f"  Barcelona rows: {len(df_socio_bcn):,}")
print(f"  Rows filtered out: {len(df_socio) - len(df_socio_bcn):,}")

# Remove "estat" column (100% null)
if 'estat' in df_socio_bcn.columns:
    df_socio_bcn = df_socio_bcn.drop(columns=['estat'])
    print(f"\n✓ Removed 'estat' column")

print(f"\nFinal df_socio_bcn shape: {df_socio_bcn.shape}")
print(f"Columns: {list(df_socio_bcn.columns)}")
print(f"\nFirst rows:")
df_socio_bcn.head()

# Save cleaned socioeconomic data
output_dir = Path("clean")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "socio_clean.parquet"
df_socio_bcn.to_parquet(output_file, index=False)
print(f"\n✓ Saved cleaned socioeconomic data to {output_file}")


## 4. Temporal and Spatial Coverage Analysis

After cleaning the data, we analyze both temporal and spatial coverage of each dataset to understand:
- **Temporal coverage**: Date ranges for each dataset and overlaps between datasets (important for joining data)
- **Spatial coverage**: Number of unique census sections in each dataset and overlaps between datasets (important for merging data)

This analysis helps identify the common time period where all three datasets (consumption, leaks, and weather) have data available.


### 4.1 Weather Station Time Coverage

Check the temporal coverage for each weather station to understand data availability. This helps identify which stations have complete data and which periods are covered by all stations.

In [ ]:
# Check time coverage for each weather station
# Show earliest and latest DATA_LECTURA for each unique station
print("Time coverage per weather station:")
print("=" * 80)

# Convert DATA_LECTURA to datetime if it's not already
df_weather['DATA_LECTURA'] = pd.to_datetime(df_weather['DATA_LECTURA'], format='%d/%m/%Y', errors='coerce')

# Group by station and get min/max dates
station_coverage = df_weather.groupby(['CODI_ESTACIO']).agg({
    'DATA_LECTURA': ['min', 'max', 'count']
}).reset_index()

# Flatten column names
station_coverage.columns = ['CODI_ESTACIO', 'Earliest_Date', 'Latest_Date', 'Total_Records']

# Calculate date range in days
station_coverage['Date_Range_Days'] = (station_coverage['Latest_Date'] - station_coverage['Earliest_Date']).dt.days

# Store datetime versions for calculations
earliest_dt = station_coverage['Earliest_Date'].min()
latest_dt = station_coverage['Latest_Date'].max()

# Format dates for display
station_coverage['Earliest_Date'] = station_coverage['Earliest_Date'].dt.strftime('%Y-%m-%d')
station_coverage['Latest_Date'] = station_coverage['Latest_Date'].dt.strftime('%Y-%m-%d')

# Sort by earliest date
station_coverage = station_coverage.sort_values('Earliest_Date')

print(f"\nTotal stations: {len(station_coverage)}")
print(f"\nTime coverage summary:")
print(station_coverage.to_string(index=False))

print(f"\n\nOverall date range:")
print(f"  Earliest: {earliest_dt.strftime('%Y-%m-%d')}")
print(f"  Latest: {latest_dt.strftime('%Y-%m-%d')}")
print(f"  Total days: {(latest_dt - earliest_dt).days}")


### 4.2 Temporal Coverage Check for Consumption and Leak Data

Check the date ranges for cleaned consumption and leak data, and compute overlaps between all three datasets (consumption, leaks, and weather). This information is crucial for determining the analysis period where all data sources are available.


In [ ]:
# Temporal check for cleaned consum data
# Check time coverage for cleaned consumption data
print("=" * 80)
print("TEMPORAL CHECK: Cleaned Consumption Data (consum)")
print("=" * 80)

import glob
from pathlib import Path

# Get all cleaned consum split files
consum_dir = Path("clean/split_consum_bcn")
consum_files = sorted(glob.glob(str(consum_dir / "consum_clean_bcn_part_*.parquet")))

print(f"\nFound {len(consum_files)} cleaned consum files to analyze")
print("-" * 80)

# Load and process each file to get date ranges (memory-efficient: track min/max only)
total_records_consum = 0
earliest_consum = None
latest_consum = None

for file_path in consum_files:
    df_chunk = pd.read_parquet(file_path)
    total_records_consum += len(df_chunk)
    
    # Convert FECHA to datetime if not already
    if 'FECHA' in df_chunk.columns:
        df_chunk['FECHA'] = pd.to_datetime(df_chunk['FECHA'], errors='coerce')
        # Get valid dates (not null)
        valid_dates = df_chunk['FECHA'].dropna()
        if len(valid_dates) > 0:
            file_min = valid_dates.min()
            file_max = valid_dates.max()
            if earliest_consum is None or file_min < earliest_consum:
                earliest_consum = file_min
            if latest_consum is None or file_max > latest_consum:
                latest_consum = file_max

if earliest_consum is not None and latest_consum is not None:
    date_range_days_consum = (latest_consum - earliest_consum).days
    
    print(f"\nOverall date range for cleaned consum:")
    print(f"  Earliest date: {earliest_consum.strftime('%Y-%m-%d')}")
    print(f"  Latest date: {latest_consum.strftime('%Y-%m-%d')}")
    print(f"  Total days: {date_range_days_consum:,}")
    print(f"  Total records: {total_records_consum:,}")
else:
    print("\n⚠ No valid dates found in cleaned consum data")

print("\n" + "=" * 80)
print("TEMPORAL CHECK: Cleaned Leak Incidents Data (fuites/avisos)")
print("=" * 80)

# Load single cleaned fuites file
fuites_file = Path("clean/fuites_clean_bcn.parquet")

if fuites_file.exists():
    print(f"\nLoading cleaned fuites data from: {fuites_file}")
    print("-" * 80)
    
    df_fuites_temp = pd.read_parquet(fuites_file)
    total_records_fuites = len(df_fuites_temp)
    
    # Convert CREATED_MENSAJE to datetime if not already
    if 'CREATED_MENSAJE' in df_fuites_temp.columns:
        df_fuites_temp['CREATED_MENSAJE'] = pd.to_datetime(df_fuites_temp['CREATED_MENSAJE'], errors='coerce')
        # Get valid dates (not null)
        valid_dates = df_fuites_temp['CREATED_MENSAJE'].dropna()
        if len(valid_dates) > 0:
            earliest_fuites = valid_dates.min()
            latest_fuites = valid_dates.max()
        else:
            earliest_fuites = None
            latest_fuites = None
    else:
        earliest_fuites = None
        latest_fuites = None
else:
    print(f"\n⚠ File not found: {fuites_file}")
    total_records_fuites = 0
    earliest_fuites = None
    latest_fuites = None

if earliest_fuites is not None and latest_fuites is not None:
    date_range_days_fuites = (latest_fuites - earliest_fuites).days
    
    print(f"\nOverall date range for cleaned fuites:")
    print(f"  Earliest date: {earliest_fuites.strftime('%Y-%m-%d')}")
    print(f"  Latest date: {latest_fuites.strftime('%Y-%m-%d')}")
    print(f"  Total days: {date_range_days_fuites:,}")
    print(f"  Total records: {total_records_fuites:,}")
else:
    print("\n⚠ No valid dates found in cleaned fuites data")

print("\n" + "=" * 80)
print("COMPARISON WITH WEATHER DATA")
print("=" * 80)

# Compute weather data range dynamically
if 'df_weather' in globals() and 'DATA_LECTURA' in df_weather.columns:
    df_weather_temp = df_weather.copy()
    df_weather_temp['DATA_LECTURA'] = pd.to_datetime(df_weather_temp['DATA_LECTURA'], format='%d/%m/%Y', errors='coerce')
    valid_dates = df_weather_temp['DATA_LECTURA'].dropna()
    if len(valid_dates) > 0:
        earliest_weather = valid_dates.min()
        latest_weather = valid_dates.max()
        date_range_days_weather = (latest_weather - earliest_weather).days
    else:
        earliest_weather = None
        latest_weather = None
        date_range_days_weather = None
else:
    earliest_weather = None
    latest_weather = None
    date_range_days_weather = None

if earliest_consum is not None and latest_consum is not None and earliest_fuites is not None and latest_fuites is not None:
    if earliest_weather is not None and latest_weather is not None:
        print(f"\nWeather data range: {earliest_weather.strftime('%Y-%m-%d')} to {latest_weather.strftime('%Y-%m-%d')} ({date_range_days_weather} days)")
    else:
        print(f"\nWeather data range: Unable to compute (df_weather not available)")
    print(f"Consum data range: {earliest_consum.strftime('%Y-%m-%d')} to {latest_consum.strftime('%Y-%m-%d')} ({date_range_days_consum} days)")
    print(f"Fuites data range: {earliest_fuites.strftime('%Y-%m-%d')} to {latest_fuites.strftime('%Y-%m-%d')} ({date_range_days_fuites} days)")
    
    # Calculate overlaps between all three datasets
    if earliest_weather is not None and latest_weather is not None:
        # Overlap between consum and fuites
        overlap_consum_fuites_start = max(earliest_consum, earliest_fuites)
        overlap_consum_fuites_end = min(latest_consum, latest_fuites)
        if overlap_consum_fuites_start <= overlap_consum_fuites_end:
            overlap_consum_fuites_days = (overlap_consum_fuites_end - overlap_consum_fuites_start).days
            print(f"\nOverlap between consum and fuites:")
            print(f"  From: {overlap_consum_fuites_start.strftime('%Y-%m-%d')}")
            print(f"  To: {overlap_consum_fuites_end.strftime('%Y-%m-%d')}")
            print(f"  Days: {overlap_consum_fuites_days:,}")
        
        # Overlap between consum and weather
        overlap_consum_weather_start = max(earliest_consum, earliest_weather)
        overlap_consum_weather_end = min(latest_consum, latest_weather)
        if overlap_consum_weather_start <= overlap_consum_weather_end:
            overlap_consum_weather_days = (overlap_consum_weather_end - overlap_consum_weather_start).days
            print(f"\nOverlap between consum and weather:")
            print(f"  From: {overlap_consum_weather_start.strftime('%Y-%m-%d')}")
            print(f"  To: {overlap_consum_weather_end.strftime('%Y-%m-%d')}")
            print(f"  Days: {overlap_consum_weather_days:,}")
        
        # Overlap between fuites and weather
        overlap_fuites_weather_start = max(earliest_fuites, earliest_weather)
        overlap_fuites_weather_end = min(latest_fuites, latest_weather)
        if overlap_fuites_weather_start <= overlap_fuites_weather_end:
            overlap_fuites_weather_days = (overlap_fuites_weather_end - overlap_fuites_weather_start).days
            print(f"\nOverlap between fuites and weather:")
            print(f"  From: {overlap_fuites_weather_start.strftime('%Y-%m-%d')}")
            print(f"  To: {overlap_fuites_weather_end.strftime('%Y-%m-%d')}")
            print(f"  Days: {overlap_fuites_weather_days:,}")
        
        # Overlap between all three (consum, fuites, and weather)
        overlap_all_start = max(earliest_consum, earliest_fuites, earliest_weather)
        overlap_all_end = min(latest_consum, latest_fuites, latest_weather)
        if overlap_all_start <= overlap_all_end:
            overlap_all_days = (overlap_all_end - overlap_all_start).days
            print(f"\nOverlap between consum, fuites, and weather:")
            print(f"  From: {overlap_all_start.strftime('%Y-%m-%d')}")
            print(f"  To: {overlap_all_end.strftime('%Y-%m-%d')}")
            print(f"  Days: {overlap_all_days:,}")
    else:
        # Only calculate overlap between consum and fuites if weather is not available
        overlap_start = max(earliest_consum, earliest_fuites)
        overlap_end = min(latest_consum, latest_fuites)
        if overlap_start <= overlap_end:
            overlap_days = (overlap_end - overlap_start).days
            print(f"\nOverlap between consum and fuites:")
            print(f"  From: {overlap_start.strftime('%Y-%m-%d')}")
            print(f"  To: {overlap_end.strftime('%Y-%m-%d')}")
            print(f"  Days: {overlap_days:,}")


### 4.3 Spatial Coverage Analysis

Analyze the spatial coverage (number of unique census sections) across all datasets (except weather) to understand data availability and assess the feasibility of merging datasets.


In [ ]:
# Spatial Coverage Analysis
# Analyze unique census sections across all datasets (except weather)

import glob
from pathlib import Path

# Load socioeconomic data
socio_file = Path("clean/socio_clean.parquet")
if socio_file.exists():
    df_socio = pd.read_parquet(socio_file)
    sections_socio = set(df_socio['SECCIO_CENSAL'].unique())
else:
    sections_socio = set()

# Load consumption data (from split files)
consum_dir = Path("clean/split_consum_bcn")
consum_files = sorted(glob.glob(str(consum_dir / "consum_clean_bcn_part_*.parquet")))
sections_consum = set()
if consum_files:
    for file_path in consum_files:
        df_chunk = pd.read_parquet(file_path)
        sections_consum.update(df_chunk['SECCIO_CENSAL'].unique())

# Load leak incidents data (from single file)
fuites_file = Path("clean/fuites_clean_bcn.parquet")
sections_fuites = set()
if fuites_file.exists():
    df_fuites_temp = pd.read_parquet(fuites_file)
    sections_fuites.update(df_fuites_temp['SECCIO_CENSAL'].unique())

# Number of census sections in each dataset
print("Number of census sections per dataset:")
print(f"  Socioeconomic: {len(sections_socio):,}")
print(f"  Consumption: {len(sections_consum):,}")
print(f"  Leaks: {len(sections_fuites):,}")

# Overlap analysis
if sections_socio and sections_consum and sections_fuites:
    total_sections = len(sections_socio)
    
    # Calculate overlaps
    sections_all_three = sections_socio & sections_consum & sections_fuites
    sections_socio_consum_only = (sections_socio & sections_consum) - sections_fuites
    sections_socio_fuites_only = (sections_socio & sections_fuites) - sections_consum
    sections_consum_fuites_only = (sections_consum & sections_fuites) - sections_socio
    sections_socio_only = sections_socio - sections_consum - sections_fuites
    sections_consum_only = sections_consum - sections_socio - sections_fuites
    sections_fuites_only = sections_fuites - sections_socio - sections_consum
    sections_all = sections_socio | sections_consum | sections_fuites
    
    # Calculate percentages
    pct_consum = (len(sections_consum) / total_sections * 100) if total_sections > 0 else 0
    pct_fuites = (len(sections_fuites) / total_sections * 100) if total_sections > 0 else 0
    pct_all_three = (len(sections_all_three) / total_sections * 100) if total_sections > 0 else 0
    pct_socio_consum = (len(sections_socio_consum_only) / total_sections * 100) if total_sections > 0 else 0
    pct_socio_fuites = (len(sections_socio_fuites_only) / total_sections * 100) if total_sections > 0 else 0
    pct_consum_fuites = (len(sections_consum_fuites_only) / total_sections * 100) if total_sections > 0 else 0
    pct_socio_only = (len(sections_socio_only) / total_sections * 100) if total_sections > 0 else 0
    pct_consum_only = (len(sections_consum_only) / total_sections * 100) if total_sections > 0 else 0
    pct_fuites_only = (len(sections_fuites_only) / total_sections * 100) if total_sections > 0 else 0
    pct_total = (len(sections_all) / total_sections * 100) if total_sections > 0 else 0
    
    print("\nOverlap analysis:")
    print(f"  All three datasets: {len(sections_all_three):,} ({pct_all_three:.1f}%)")
    print(f"  Socioeconomic + Consumption only: {len(sections_socio_consum_only):,} ({pct_socio_consum:.1f}%)")
    print(f"  Socioeconomic + Leaks only: {len(sections_socio_fuites_only):,} ({pct_socio_fuites:.1f}%)")
    print(f"  Consumption + Leaks only: {len(sections_consum_fuites_only):,} ({pct_consum_fuites:.1f}%)")
    print(f"  Socioeconomic only: {len(sections_socio_only):,} ({pct_socio_only:.1f}%)")
    print(f"  Consumption only: {len(sections_consum_only):,} ({pct_consum_only:.1f}%)")
    print(f"  Leaks only: {len(sections_fuites_only):,} ({pct_fuites_only:.1f}%)")
    print(f"  Total unique sections: {len(sections_all):,} ({pct_total:.1f}%)")
    
    # Summary table
    coverage_summary = pd.DataFrame({
        'Dataset': ['Socioeconomic (Total)', 'Consumption', 'Leaks', 'All Three', 'Total Unique'],
        'Unique Sections': [
            len(sections_socio),
            len(sections_consum),
            len(sections_fuites),
            len(sections_all_three),
            len(sections_all)
        ],
        'Coverage %': [100.0, pct_consum, pct_fuites, pct_all_three, pct_total]
    })
    
    print("\nSummary:")
    print(coverage_summary.to_string(index=False))
    
elif sections_socio and sections_consum:
    total_sections = len(sections_socio)
    sections_both = sections_socio & sections_consum
    sections_socio_only = sections_socio - sections_consum
    sections_consum_only = sections_consum - sections_socio
    
    pct_both = (len(sections_both) / total_sections * 100) if total_sections > 0 else 0
    pct_socio_only = (len(sections_socio_only) / total_sections * 100) if total_sections > 0 else 0
    pct_consum_only = (len(sections_consum_only) / total_sections * 100) if total_sections > 0 else 0
    pct_total = (len(sections_socio | sections_consum) / total_sections * 100) if total_sections > 0 else 0
    pct_consum = (len(sections_consum) / total_sections * 100) if total_sections > 0 else 0
    
    print("\nOverlap analysis:")
    print(f"  Both datasets: {len(sections_both):,} ({pct_both:.1f}%)")
    print(f"  Socioeconomic only: {len(sections_socio_only):,} ({pct_socio_only:.1f}%)")
    print(f"  Consumption only: {len(sections_consum_only):,} ({pct_consum_only:.1f}%)")
    print(f"  Total unique sections: {len(sections_socio | sections_consum):,} ({pct_total:.1f}%)")
    
    coverage_summary = pd.DataFrame({
        'Dataset': ['Socioeconomic (Total)', 'Consumption', 'Both', 'Total Unique'],
        'Unique Sections': [
            len(sections_socio),
            len(sections_consum),
            len(sections_both),
            len(sections_socio | sections_consum)
        ],
        'Coverage %': [100.0, pct_consum, pct_both, pct_total]
    })
    
    print("\nSummary:")
    print(coverage_summary.to_string(index=False))
else:
    print("⚠ Need at least two datasets to perform overlap analysis")


**Why data merging is not recommended:**

Based on the spatial coverage analysis above, merging the datasets would result in significant data loss:

- **Only 29.8% of census sections** (318 out of 1,068) have data in all three datasets
- **41.9% of sections** (447) would have missing consumption data
- **59.2% of sections** (632) would have missing leak data
- **30.8% of sections** (329) would have missing data in both consumption and leaks

If we merge the datasets using socioeconomic data as the base (left join), 70.2% of the rows would have `NaN` values for one or more variables. This would:
- Reduce the usable sample size for any analysis requiring all three variables
- Potentially introduce bias (sections with complete data may not be representative)
- Require imputation or exclusion strategies that could distort results

**Recommendation:** Keep datasets separate and join on-demand for specific analyses that can accept the reduced sample size (318 sections with all three variables) or work with incomplete data.


In [ ]:
# Preview cleaned datasets
# Display sample rows from each cleaned dataset to verify data structure

import glob
from pathlib import Path

print("=" * 80)
print("PREVIEW OF CLEANED DATASETS")
print("=" * 80)

# 1. Consumption data (first split file)
print("\n1. CONSUMPTION DATA (first split file):")
print("-" * 80)
consum_dir = Path("clean/split_consum_bcn")
consum_files = sorted(glob.glob(str(consum_dir / "consum_clean_bcn_part_*.parquet")))
if consum_files:
    df_consum_sample = pd.read_parquet(consum_files[0])
    print(f"File: {Path(consum_files[0]).name}")
    print(f"Shape: {df_consum_sample.shape}")
    print(f"Columns: {list(df_consum_sample.columns)}")
    print("\nFirst 5 rows:")
    print(df_consum_sample.head())
else:
    print("⚠ No consumption split files found")

# 2. Leak incidents data (first split file)
print("\n\n2. LEAK INCIDENTS DATA:")
print("-" * 80)
fuites_file = Path("clean/fuites_clean_bcn.parquet")
if fuites_file.exists():
    df_fuites_sample = pd.read_parquet(fuites_file)
    print(f"File: {fuites_file.name}")
    print(f"Shape: {df_fuites_sample.shape}")
    print(f"Columns: {list(df_fuites_sample.columns)}")
    print("\nFirst 5 rows:")
    print(df_fuites_sample.head())
else:
    print("⚠ Leak incidents file not found")

# 3. Weather data
print("\n\n3. WEATHER DATA:")
print("-" * 80)
weather_file = Path("clean/weather_clean.parquet")
if weather_file.exists():
    df_weather_sample = pd.read_parquet(weather_file)
    print(f"File: {weather_file.name}")
    print(f"Shape: {df_weather_sample.shape}")
    print(f"Columns: {list(df_weather_sample.columns)}")
    print("\nFirst 5 rows:")
    print(df_weather_sample.head())
else:
    print("⚠ Weather cleaned file not found")

# 4. Socioeconomic data
print("\n\n4. SOCIOECONOMIC DATA:")
print("-" * 80)
socio_file = Path("clean/socio_clean.parquet")
if socio_file.exists():
    df_socio_sample = pd.read_parquet(socio_file)
    print(f"File: {socio_file.name}")
    print(f"Shape: {df_socio_sample.shape}")
    print(f"Columns: {list(df_socio_sample.columns)}")
    print("\nFirst 5 rows:")
    print(df_socio_sample.head())
else:
    print("⚠ Socioeconomic cleaned file not found")

print("\n" + "=" * 80)


## 5. Exploratory Data Analysis (EDA)

Perform detailed exploratory data analysis on the cleaned datasets to understand patterns, distributions, and relationships in the data. We work directly with the cleaned data files, performing any necessary aggregations inline for specific visualizations.


We will work directly with the cleaned data files:
- **Consumption data**: Split parquet files in `clean/split_consum_bcn/`
- **Leak incidents data**: Single parquet file `clean/fuites_clean_bcn.parquet`
- **Weather data**: `clean/weather_clean.parquet`
- **Socioeconomic data**: `clean/socio_clean.parquet`

Any aggregations needed for specific visualizations will be performed inline.

### 5.1 Socioeconomic Analysis

Analyze the distribution of IST (Índex Socioeconòmic Territorial) across Barcelona census sections.

In [ ]:
# Load socioeconomic data if not already loaded
if 'df_socio_bcn' not in globals():
    df_socio_bcn = pd.read_parquet("clean/socio_clean.parquet")

# Ensure 'valor' column is numeric
if df_socio_bcn['valor'].dtype == object:
    df_socio_bcn['valor'] = pd.to_numeric(df_socio_bcn['valor'], errors='coerce')

print("Available columns in socioeconomic data:")
print(df_socio_bcn.columns.tolist())
print(f"\nData shape: {df_socio_bcn.shape}")
print(f"\nSample data:")
print(df_socio_bcn.head())

print("\nDescriptive statistics for socio-economic index (valor):")
print(df_socio_bcn['valor'].describe())

plt.figure(figsize=(10, 5))
sns.histplot(df_socio_bcn['valor'], bins=30, color='teal', kde=True)
plt.title("Distribution of Socioeconomic Index")
plt.xlabel("Index value")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Box plot for better understanding of distribution
plt.figure(figsize=(10, 5))
sns.boxplot(y=df_socio_bcn['valor'], color='teal')
plt.title("Box Plot of Socioeconomic Index")
plt.ylabel("Index value")
plt.tight_layout()
plt.show()

# Top/bottom census sections by socio-economic index
top_sections = df_socio_bcn.sort_values('valor', ascending=False).head(10)
bottom_sections = df_socio_bcn.sort_values('valor').head(10)

print("\nTop 10 census sections by socio-economic index:")
print(top_sections[['any', 'SECCIO_CENSAL', 'valor']])

print("\nBottom 10 census sections by socio-economic index:")
print(bottom_sections[['any', 'SECCIO_CENSAL', 'valor']])


### 5.2 Consumption Data Analysis

Analyze the distribution of daily water consumption across census sections.

In [ ]:
# Load and aggregate consumption data from cleaned split files
# Aggregate by SECCIO_CENSAL and FECHA (sum CONSUMO_REAL per section per day)
import glob
from pathlib import Path

print("CONSUMPTION DATA ANALYSIS")
print("-" * 80)

consum_dir = Path("clean/split_consum_bcn")
consum_files = sorted(glob.glob(str(consum_dir / "consum_clean_bcn_part_*.parquet")))

print(f"Found {len(consum_files)} cleaned consumption files")

# Process all files and aggregate
consum_agg_list = []
total_rows = 0

for i, file_path in enumerate(consum_files, 1):
    if i % 5 == 0 or i == len(consum_files):
        print(f"Processing file {i}/{len(consum_files)}: {Path(file_path).name}")
    
    df_chunk = pd.read_parquet(file_path)
    df_chunk['FECHA'] = pd.to_datetime(df_chunk['FECHA'], errors='coerce')
    total_rows += len(df_chunk)
    
    # Aggregate: sum CONSUMO_REAL by SECCIO_CENSAL and FECHA
    df_chunk_agg = (
        df_chunk
        .groupby(['SECCIO_CENSAL', 'FECHA'])
        .agg({'CONSUMO_REAL': 'sum'})
        .reset_index()
    )
    
    consum_agg_list.append(df_chunk_agg)

# Combine all aggregated chunks
df_consum_agg = pd.concat(consum_agg_list, ignore_index=True)

# Final aggregation (in case same SECCIO_CENSAL + FECHA appears in multiple files)
df_consum_agg = (
    df_consum_agg
    .groupby(['SECCIO_CENSAL', 'FECHA'])
    .agg({'CONSUMO_REAL': 'sum'})
    .reset_index()
)

print(f"\n✓ Processed {total_rows:,} rows from {len(consum_files)} files")
print(f"✓ Aggregated to {len(df_consum_agg):,} section-date combinations")
print(f"Date range: {df_consum_agg['FECHA'].min()} to {df_consum_agg['FECHA'].max()}")
print(f"Unique census sections: {df_consum_agg['SECCIO_CENSAL'].nunique():,}")
print(f"Unique dates: {df_consum_agg['FECHA'].nunique():,}")

print(f"\nConsumption statistics (daily per census section):")
print(df_consum_agg['CONSUMO_REAL'].describe())

# 1. Distribution plot: Histogram of average consumption per census section
avg_consumption_per_section = (
    df_consum_agg
    .groupby('SECCIO_CENSAL')['CONSUMO_REAL']
    .mean()
    .reset_index(name='avg_consumption')
)

# Get data range for axis limits
data_min = avg_consumption_per_section['avg_consumption'].min()
data_max = avg_consumption_per_section['avg_consumption'].max()
data_range = data_max - data_min
# Add 5% padding on each side
x_min = data_min - (data_range * 0.05)
x_max = data_max + (data_range * 0.05)

plt.figure(figsize=(12, 6))
sns.histplot(avg_consumption_per_section['avg_consumption'], bins=50, color='skyblue', kde=True)
plt.title("Distribution of Average Daily Water Consumption per Census Section")
plt.xlabel("Average Daily Consumption per Census Section (liters/day)")
plt.ylabel("Number of Census Sections")
plt.xlim(x_min, x_max)
plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

print(f"\nAverage consumption per section statistics:")
print(avg_consumption_per_section['avg_consumption'].describe())

# 2. Box plot: Summary statistics
plt.figure(figsize=(10, 6))
sns.boxplot(y=df_consum_agg['CONSUMO_REAL'], color='lightblue')
plt.yscale('log')
plt.title("Box Plot: Summary Statistics of Daily Water Consumption per Census Section")
plt.ylabel("Daily Consumption (liters/day)")
plt.tight_layout()
plt.show()

# 3. Consumption by month
df_consum_agg['month'] = df_consum_agg['FECHA'].dt.month
df_consum_agg['month_name'] = df_consum_agg['FECHA'].dt.strftime('%B')

consumption_by_month = (
    df_consum_agg
    .groupby('month')['CONSUMO_REAL']
    .agg(['sum', 'mean', 'count'])
    .reset_index()
)
consumption_by_month['month_name'] = pd.to_datetime(consumption_by_month['month'], format='%m').dt.strftime('%B')
consumption_by_month = consumption_by_month.sort_values('month')

print("\nTotal consumption by month:")
print(consumption_by_month[['month_name', 'sum', 'mean', 'count']])

# Plot total consumption by month
plt.figure(figsize=(12, 6))
sns.barplot(data=consumption_by_month, x='month_name', y='sum', color='skyblue', order=consumption_by_month['month_name'])
plt.title("Total Water Consumption by Month")
plt.xlabel("Month")
plt.ylabel("Total Consumption (liters)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 4. Consumption over time (daily aggregate)
consumption_daily = (
    df_consum_agg
    .groupby('FECHA')['CONSUMO_REAL']
    .agg(['sum', 'mean'])
    .reset_index()
    .sort_values('FECHA')
)

plt.figure(figsize=(14, 6))
plt.plot(consumption_daily['FECHA'], consumption_daily['sum'], color='steelblue', linewidth=1, alpha=0.7)
plt.title("Total Daily Water Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Total Consumption (liters/day)")
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 5. Average consumption per census section over time (top 10 sections by total consumption)
section_totals = (
    df_consum_agg
    .groupby('SECCIO_CENSAL')['CONSUMO_REAL']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

top_sections = section_totals.index.tolist()
df_top_sections = df_consum_agg[df_consum_agg['SECCIO_CENSAL'].isin(top_sections)]

plt.figure(figsize=(14, 6))
for section in top_sections:
    section_data = df_top_sections[df_top_sections['SECCIO_CENSAL'] == section].sort_values('FECHA')
    plt.plot(section_data['FECHA'], section_data['CONSUMO_REAL'], label=section, linewidth=1, alpha=0.7)

plt.title("Daily Water Consumption Over Time (Top 10 Census Sections by Total Consumption)")
plt.xlabel("Date")
plt.ylabel("Daily Consumption (liters/day)")
plt.xticks(rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 5.3 Leak Incidents Analysis

Analyze temporal patterns in leak incidents across Barcelona. Since the cleaned data has one leak per row (one leak per POLISSA_SUBM per day), we can analyze the data directly without additional aggregation steps. The analysis includes:

- Distribution of leaks per census section-date
- Temporal trends (daily, monthly)
- Analysis by use type (US_AIGUA_SUBM) 

In [ ]:
# Load leak incidents data from cleaned single parquet file
# Since we have one leak per row (one leak per POLISSA_SUBM per day), we can work directly with the data
from pathlib import Path

print("=" * 80)
print("LEAK INCIDENTS ANALYSIS")
print("=" * 80)

# Load single cleaned parquet file
fuites_file = Path("clean/fuites_clean_bcn.parquet")

if fuites_file.exists():
    print(f"\nLoading cleaned leak data from: {fuites_file}")
    print("-" * 80)
    
    df_fuites = pd.read_parquet(fuites_file)
    # CREATED_MENSAJE is already a date (from cleaning step), convert to datetime for analysis
    df_fuites['FECHA'] = pd.to_datetime(df_fuites['CREATED_MENSAJE'])
else:
    print(f"\n⚠ Error: File not found: {fuites_file}")
    print("Please run the cleaning step first (Section 3.2)")
    df_fuites = pd.DataFrame()

if len(df_fuites) > 0:
    print(f"\n✓ Loaded {len(df_fuites):,} leaks from single file")
    print(f"Date range: {df_fuites['FECHA'].min()} to {df_fuites['FECHA'].max()}")
    print(f"Unique census sections: {df_fuites['SECCIO_CENSAL'].nunique():,}")
    print(f"Unique dates: {df_fuites['FECHA'].nunique():,}")
    print(f"Unique POLISSA_SUBM: {df_fuites['POLISSA_SUBM'].nunique():,}")
else:
    print(f"\n⚠ No data loaded - cannot continue analysis")

# Aggregate by section-date for analysis
df_fuites_agg = (
    df_fuites
    .groupby(['SECCIO_CENSAL', 'FECHA'])
    .size()
    .reset_index(name='num_fuites')
)

print(f"\nLeak statistics (number of leaks per section-date):")
print(df_fuites_agg['num_fuites'].describe())

# 1. Distribution of leaks per POLISSA_SUBM (how many leaks does each policy have?)
leaks_per_polissa = (
    df_fuites
    .groupby('POLISSA_SUBM')
    .size()
    .reset_index(name='num_leaks')
    .sort_values('num_leaks', ascending=False)
)

print(f"\nLeak statistics per POLISSA_SUBM:")
print(leaks_per_polissa['num_leaks'].describe())

# Count how many policies have each number of leaks
leak_counts_polissa = leaks_per_polissa['num_leaks'].value_counts().sort_index()

plt.figure(figsize=(14, 6))
leak_counts_polissa.plot(kind='bar', color='coral', width=0.8)
plt.title("Distribution of Leak Frequency per Policy (POLISSA_SUBM)")
plt.xlabel("Number of Leaks per Policy")
plt.ylabel("Number of Policies")
plt.xticks(rotation=0, ha='center')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# 2. Distribution of leaks per SECCIO_CENSAL (how many leaks per census section?)
leaks_per_section = (
    df_fuites
    .groupby('SECCIO_CENSAL')
    .size()
    .reset_index(name='num_leaks')
    .sort_values('num_leaks', ascending=False)
)

print(f"\nLeak statistics per SECCIO_CENSAL:")
print(leaks_per_section['num_leaks'].describe())

# Count how many sections have each number of leaks
leak_counts_section = leaks_per_section['num_leaks'].value_counts().sort_index()

plt.figure(figsize=(14, 6))
leak_counts_section.plot(kind='bar', color='coral', width=0.8)
plt.title("Distribution of Leak Frequency per Census Section (SECCIO_CENSAL)")
plt.xlabel("Number of Leaks per Census Section")
plt.ylabel("Number of Census Sections")
plt.xticks(rotation=0, ha='center')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# Additional: Top sections and policies by leak count
print(f"\nTop 10 census sections by number of leaks:")
print(leaks_per_section.head(10))

print(f"\nTop 10 policies by number of leaks:")
print(leaks_per_polissa.head(10))

# 3. Leaks over time (daily aggregate)
leaks_daily = (
    df_fuites
    .groupby('FECHA')
    .size()
    .reset_index(name='num_fuites')
    .sort_values('FECHA')
)

plt.figure(figsize=(14, 6))
plt.plot(leaks_daily['FECHA'], leaks_daily['num_fuites'], color='teal', linewidth=1, alpha=0.7)
plt.title("Total Daily Leaks in Barcelona Over Time")
plt.xlabel("Date")
plt.ylabel("Total Number of Leaks per Day")
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 4. Leaks by month (only full years: 2023 and 2024)
# Filter to only include full years to avoid partial year bias
df_fuites['year'] = df_fuites['FECHA'].dt.year
df_fuites_full_years = df_fuites[df_fuites['year'].isin([2023, 2024])].copy()

print(f"\nFiltering to full years only (2023 and 2024):")
print(f"  Total leaks in dataset: {len(df_fuites):,}")
print(f"  Leaks in 2023-2024: {len(df_fuites_full_years):,}")
print(f"  Leaks excluded (partial years): {len(df_fuites) - len(df_fuites_full_years):,}")

df_fuites_full_years['month'] = df_fuites_full_years['FECHA'].dt.month
df_fuites_full_years['month_name'] = df_fuites_full_years['FECHA'].dt.strftime('%B')

leaks_by_month = (
    df_fuites_full_years
    .groupby('month')
    .size()
    .reset_index(name='total_leaks')
)
leaks_by_month['month_name'] = pd.to_datetime(leaks_by_month['month'], format='%m').dt.strftime('%B')
leaks_by_month = leaks_by_month.sort_values('month')

# Also calculate mean leaks per section-date by month (using full years only)
df_fuites_agg_full_years = df_fuites_agg[df_fuites_agg['FECHA'].dt.year.isin([2023, 2024])].copy()
leaks_by_month_agg = (
    df_fuites_agg_full_years
    .assign(month=df_fuites_agg_full_years['FECHA'].dt.month)
    .groupby('month')['num_fuites']
    .agg(['mean', 'count'])
    .reset_index()
)
leaks_by_month = leaks_by_month.merge(leaks_by_month_agg, on='month', how='left')
leaks_by_month = leaks_by_month.sort_values('month')

print("\nTotal leaks by month (2023-2024 only, full years):")
print(leaks_by_month[['month_name', 'total_leaks', 'mean', 'count']])

plt.figure(figsize=(12, 6))
sns.barplot(data=leaks_by_month, x='month_name', y='total_leaks', color='coral', order=leaks_by_month['month_name'])
plt.title("Total Leaks by Month (2023-2024, Full Years Only)")
plt.xlabel("Month")
plt.ylabel("Total Number of Leaks")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 5. Leaks by use type (US_AIGUA_SUBM)
if 'US_AIGUA_SUBM' in df_fuites.columns:
    leaks_by_use = (
        df_fuites
        .groupby('US_AIGUA_SUBM')
        .size()
        .reset_index(name='num_leaks')
        .sort_values('num_leaks', ascending=False)
    )
    
    print("\nLeaks by use type:")
    print(leaks_by_use)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=leaks_by_use, x='US_AIGUA_SUBM', y='num_leaks', color='coral')
    plt.title("Total Leaks by Use Type")
    plt.xlabel("Use Type (US_AIGUA_SUBM)")
    plt.ylabel("Total Number of Leaks")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Leaks by use type over time
    leaks_by_use_time = (
        df_fuites
        .groupby(['FECHA', 'US_AIGUA_SUBM'])
        .size()
        .reset_index(name='num_leaks')
        .sort_values('FECHA')
    )
    
    plt.figure(figsize=(14, 6))
    for use_type in leaks_by_use_time['US_AIGUA_SUBM'].unique():
        use_data = leaks_by_use_time[leaks_by_use_time['US_AIGUA_SUBM'] == use_type]
        plt.plot(use_data['FECHA'], use_data['num_leaks'], label=use_type, linewidth=1, alpha=0.7)
    plt.title("Daily Leaks by Use Type Over Time")
    plt.xlabel("Date")
    plt.ylabel("Number of Leaks per Day")
    plt.legend(title='Use Type')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠ US_AIGUA_SUBM column not found in data")


### 5.4 Consumption Patterns: Weekday vs. Weekend by Type of Use

Analyze differences in consumption patterns between weekdays and weekends, broken down by water use type.

In [ ]:
# For this analysis, we need the detailed consumption data (not aggregated)
# Load all files to analyze weekday/weekend patterns
print("Loading all consumption data for weekday/weekend analysis...")

consum_dir = Path("clean/split_consum_bcn")
consum_files = sorted(glob.glob(str(consum_dir / "consum_clean_bcn_part_*.parquet")))

print(f"Found {len(consum_files)} consumption files to load")
print("-" * 80)

# Load all files
consum_list = []
for i, file_path in enumerate(consum_files, 1):
    print(f"Loading file {i}/{len(consum_files)}: {Path(file_path).name}")
    df_chunk = pd.read_parquet(file_path)
    consum_list.append(df_chunk)

# Combine all data
df_consum_sample = pd.concat(consum_list, ignore_index=True)

print(f"\n✓ All data loaded: {len(df_consum_sample):,} rows")
print(f"Date range: {df_consum_sample['FECHA'].min()} to {df_consum_sample['FECHA'].max()}")

# Ensure date is parsed
df_consum_sample['FECHA'] = pd.to_datetime(df_consum_sample['FECHA'], errors='coerce')

# Extract weekday info
df_consum_sample['weekday'] = df_consum_sample['FECHA'].dt.day_name()
df_consum_sample['is_weekend'] = df_consum_sample['FECHA'].dt.dayofweek >= 5  # Saturday=5, Sunday=6

# Check if US_AIGUA_GEST column exists
if 'US_AIGUA_GEST' in df_consum_sample.columns:
    # Plot: Consumption by weekend and by type of use
    plt.figure(figsize=(12, 6))
    sns.boxplot(
        data=df_consum_sample,
        x='is_weekend',
        y='CONSUMO_REAL',
        hue='US_AIGUA_GEST',
        showfliers=False,
        palette='Set2'
    )
    plt.yscale('log')  # Because consumption values can vary a lot
    plt.title("Water Consumption: Weekday vs Weekend by Type of Use")
    plt.xlabel("Weekend (True = Saturday/Sunday)")
    plt.ylabel("Consumption (liters/day)")
    plt.legend(title="Use Type", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nSummary statistics by weekend and use type:")
    summary = df_consum_sample.groupby(['is_weekend', 'US_AIGUA_GEST'])['CONSUMO_REAL'].describe()
    print(summary)
else:
    print("⚠ Column 'US_AIGUA_GEST' not found in consumption data")
    print("Available columns:", df_consum_sample.columns.tolist())

### 5.5 Weather Data Analysis

Analyze weather patterns and their relationship with water consumption and leaks. We manually cleaned the column NOM_VARIABLE of the dataset to remove corrupted characters due to accents.

In [ ]:
# Load cleaned weather data
df_weather_clean = pd.read_parquet("clean/weather_clean.parquet")

# Convert DATA_LECTURA to datetime
df_weather_clean['DATA_LECTURA'] = pd.to_datetime(df_weather_clean['DATA_LECTURA'], format="%d/%m/%Y", errors='coerce')

# Identify rows with incorrect NOM_VARIABLE values
print("=" * 80)
print("ROWS WITH INCORRECT NOM_VARIABLE VALUES")
print("=" * 80)

# List of correct NOM_VARIABLE values
correct_vars = [
    'Temperatura mitjana diària',
    'Temperatura màxima diària + hora',
    'Temperatura mínima diària + hora',
    'Temperatura mitjana diària clàssica',
    'Amplitud tèrmica diària',
    'Humitat relativa mitjana diària',
    'Humitat relativa màxima diària + data',
    'Humitat relativa mínima diària + data',
    'Pressió atmosfèrica mitjana diària',
    'Pressió atmosfèrica màxima diària + hora',
    'Pressió atmosfèrica mínima diària + hora',
    'Precipitació acumulada diària',
    'Precipitació acumulada diària (8-8 h)',
    'Precipitació màxima en 1 min (diària) + hora',
    'Precipitació màxima en 1 h (diària) + hora',
    'Precipitació màxima en 30 min (diària)+ hora',
    'Irradiació solar global diària',
    'Velocitat mitjana diària del vent 10 m (esc.)',
    'Direcció mitjana diària del vent 10 m (m. 1)',
    'Ratxa màxima diària del vent 10 m + hora',
    'Direcció de la ratxa màx. diària de vent 10 m',
    'Evapotranspiració de referència',
    'Velocitat mitjana diària del vent 10 m (vec.)',
    'Direcció mitjana diària del vent 10 m (m. u)'
]

# Filter rows where NOM_VARIABLE is NOT in the correct list
if 'NOM_VARIABLE' in df_weather_clean.columns:
    incorrect_mask = ~df_weather_clean['NOM_VARIABLE'].isin(correct_vars)
    df_incorrect = df_weather_clean[incorrect_mask].copy()
    
    print(f"\nTotal rows with incorrect NOM_VARIABLE: {len(df_incorrect):,}")
    print(f"Total rows in dataset: {len(df_weather_clean):,}")
    print(f"Percentage incorrect: {len(df_incorrect)/len(df_weather_clean)*100:.4f}%")
    
    if len(df_incorrect) > 0:
        print(f"\nIncorrect variable names found:")
        incorrect_counts = df_incorrect['NOM_VARIABLE'].value_counts().sort_index()
        for var, count in incorrect_counts.items():
            print(f"  - '{var}': {count} records")
        
        print(f"\nAll rows with incorrect NOM_VARIABLE:")
        print(df_incorrect.to_string())
        
        print(f"\nSummary by incorrect variable:")
        summary = df_incorrect.groupby('NOM_VARIABLE').size().reset_index(name='count').sort_values('NOM_VARIABLE')
        print(summary.to_string(index=False))
    else:
        print("\n✓ All rows have correct NOM_VARIABLE values")
else:
    print("⚠ Column 'NOM_VARIABLE' not found in weather data")

print("\n" + "=" * 80)

# Ensure numeric 'VALOR_NUM' column is present
if 'VALOR_NUM' not in df_weather_clean.columns and 'VALOR' in df_weather_clean.columns:
    df_weather_clean['VALOR_NUM'] = (
        df_weather_clean['VALOR']
        .astype(str)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    df_weather_clean['VALOR_NUM'] = pd.to_numeric(df_weather_clean['VALOR_NUM'], errors='coerce')

print(f"Weather data shape: {df_weather_clean.shape}")
print(f"Date range: {df_weather_clean['DATA_LECTURA'].min()} to {df_weather_clean['DATA_LECTURA'].max()}")
print(f"Available columns: {df_weather_clean.columns.tolist()}")

# Focus on key temperature variables
variable_map = {
    'Temperatura màxima diària + hora': 'temp_max',
    'Temperatura mínima diària + hora': 'temp_min',
    'Temperatura mitjana diària': 'temp_avg'
}

# Create comprehensive weather analysis with multiple variables
print("\n" + "=" * 80)
print("COMPREHENSIVE WEATHER DATA ANALYSIS")
print("=" * 80)

# Pivot all weather variables to wide format for analysis
weather_pivot = (
    df_weather_clean
    .pivot_table(
        index='DATA_LECTURA', 
        columns='NOM_VARIABLE', 
        values='VALOR_NUM', 
        aggfunc='mean'
    )
    .reset_index()
    .sort_values('DATA_LECTURA')
)

print(f"\nPivoted weather data (all variables): {weather_pivot.shape}")
print(f"Variables: {list(weather_pivot.columns[1:])}")

# 1. Temperature, Humidity, and Pressure over time (multi-panel)
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Temperature
temp_vars = ['Temperatura màxima diària + hora', 'Temperatura mínima diària + hora', 'Temperatura mitjana diària']
for var in temp_vars:
    if var in weather_pivot.columns:
        axes[0].plot(weather_pivot['DATA_LECTURA'], weather_pivot[var], label=var, linewidth=0.8, alpha=0.7)
axes[0].set_ylabel('Temperature (°C)')
axes[0].set_title('Temperature Trends Over Time')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

# Humidity
humidity_vars = ['Humitat relativa màxima diària + data', 'Humitat relativa mínima diària + data', 'Humitat relativa mitjana diària']
for var in humidity_vars:
    if var in weather_pivot.columns:
        axes[1].plot(weather_pivot['DATA_LECTURA'], weather_pivot[var], label=var, linewidth=0.8, alpha=0.7)
axes[1].set_ylabel('Humidity (%)')
axes[1].set_title('Humidity Trends Over Time')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

# Pressure
pressure_vars = ['Pressió atmosfèrica màxima diària + hora', 'Pressió atmosfèrica mínima diària + hora', 'Pressió atmosfèrica mitjana diària']
for var in pressure_vars:
    if var in weather_pivot.columns:
        axes[2].plot(weather_pivot['DATA_LECTURA'], weather_pivot[var], label=var, linewidth=0.8, alpha=0.7)
axes[2].set_ylabel('Pressure (hPa)')
axes[2].set_xlabel('Date')
axes[2].set_title('Atmospheric Pressure Trends Over Time')
axes[2].legend(fontsize=8)
axes[2].grid(True, alpha=0.3)
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 2. Seasonal patterns - Monthly averages for key variables
weather_pivot['year'] = weather_pivot['DATA_LECTURA'].dt.year
weather_pivot['month'] = weather_pivot['DATA_LECTURA'].dt.month
weather_pivot['month_name'] = weather_pivot['DATA_LECTURA'].dt.strftime('%B')

# Monthly averages for temperature
monthly_temp = weather_pivot.groupby('month').agg({
    'Temperatura màxima diària + hora': 'mean',
    'Temperatura mínima diària + hora': 'mean',
    'Temperatura mitjana diària': 'mean'
}).reset_index()
monthly_temp['month_name'] = pd.to_datetime(monthly_temp['month'], format='%m').dt.strftime('%B')
monthly_temp = monthly_temp.sort_values('month')

fig, ax = plt.subplots(figsize=(12, 6))
x = range(len(monthly_temp))
width = 0.25
if 'Temperatura màxima diària + hora' in monthly_temp.columns:
    ax.bar([i - width for i in x], monthly_temp['Temperatura màxima diària + hora'], width, label='Max', color='tomato', alpha=0.8)
if 'Temperatura mitjana diària' in monthly_temp.columns:
    ax.bar(x, monthly_temp['Temperatura mitjana diària'], width, label='Average', color='seagreen', alpha=0.8)
if 'Temperatura mínima diària + hora' in monthly_temp.columns:
    ax.bar([i + width for i in x], monthly_temp['Temperatura mínima diària + hora'], width, label='Min', color='royalblue', alpha=0.8)

ax.set_xlabel('Month')
ax.set_ylabel('Temperature (°C)')
ax.set_title('Average Monthly Temperature Patterns')
ax.set_xticks(x)
ax.set_xticklabels(monthly_temp['month_name'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# 3. Precipitation analysis
if 'Precipitació acumulada diària' in weather_pivot.columns:
    precip_data = weather_pivot[['DATA_LECTURA', 'Precipitació acumulada diària']].dropna()
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Daily precipitation
    axes[0].plot(precip_data['DATA_LECTURA'], precip_data['Precipitació acumulada diària'], 
                 color='steelblue', linewidth=0.5, alpha=0.6)
    axes[0].set_ylabel('Daily Precipitation (mm)')
    axes[0].set_title('Daily Precipitation Over Time')
    axes[0].grid(True, alpha=0.3)
    
    # Monthly total precipitation
    precip_data['year'] = precip_data['DATA_LECTURA'].dt.year
    precip_data['month'] = precip_data['DATA_LECTURA'].dt.month
    monthly_precip = precip_data.groupby(['year', 'month'])['Precipitació acumulada diària'].sum().reset_index()
    monthly_precip['date'] = pd.to_datetime(monthly_precip[['year', 'month']].assign(day=1))
    monthly_precip = monthly_precip.sort_values('date')
    
    axes[1].bar(monthly_precip['date'], monthly_precip['Precipitació acumulada diària'], 
                color='steelblue', alpha=0.7, width=20)
    axes[1].set_ylabel('Monthly Total Precipitation (mm)')
    axes[1].set_xlabel('Date')
    axes[1].set_title('Monthly Total Precipitation')
    axes[1].grid(True, alpha=0.3, axis='y')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("\nPrecipitation statistics:")
    print(precip_data['Precipitació acumulada diària'].describe())

# 4. Wind analysis
wind_vars = ['Velocitat mitjana diària del vent 10 m (esc.)', 'Ratxa màxima diària del vent 10 m + hora']
wind_data = weather_pivot[['DATA_LECTURA'] + [v for v in wind_vars if v in weather_pivot.columns]].dropna()

if len(wind_data.columns) > 1:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    for i, var in enumerate([v for v in wind_vars if v in weather_pivot.columns]):
        axes[i].plot(wind_data['DATA_LECTURA'], wind_data[var], linewidth=0.8, alpha=0.7, color='darkgreen')
        axes[i].set_ylabel('Wind Speed (m/s)')
        axes[i].set_title(var)
        axes[i].grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Date')
    axes[-1].tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.show()

# 5. Distribution of key variables
key_vars_for_dist = [
    'Temperatura mitjana diària',
    'Humitat relativa mitjana diària',
    'Pressió atmosfèrica mitjana diària',
    'Precipitació acumulada diària'
]

dist_data = weather_pivot[['DATA_LECTURA'] + [v for v in key_vars_for_dist if v in weather_pivot.columns]].dropna()

if len(dist_data.columns) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for i, var in enumerate([v for v in key_vars_for_dist if v in weather_pivot.columns]):
        if i < len(axes):
            dist_data[var].hist(bins=50, ax=axes[i], color='coral', alpha=0.7, edgecolor='black')
            axes[i].set_xlabel(var)
            axes[i].set_ylabel('Frequency')
            axes[i].set_title(f'Distribution of {var}')
            axes[i].grid(True, alpha=0.3, axis='y')
    
    # Hide unused subplots
    for i in range(len([v for v in key_vars_for_dist if v in weather_pivot.columns]), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# 6. Correlation heatmap between weather variables
correlation_vars = [
    'Temperatura mitjana diària',
    'Temperatura màxima diària + hora',
    'Temperatura mínima diària + hora',
    'Humitat relativa mitjana diària',
    'Pressió atmosfèrica mitjana diària',
    'Precipitació acumulada diària',
    'Irradiació solar global diària',
    'Velocitat mitjana diària del vent 10 m (esc.)'
]

corr_data = weather_pivot[[v for v in correlation_vars if v in weather_pivot.columns]].dropna()

if len(corr_data.columns) > 1:
    corr_matrix = corr_data.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Weather Variables')
    plt.tight_layout()
    plt.show()

# 7. Box plots by month for temperature
if 'Temperatura mitjana diària' in weather_pivot.columns:
    temp_monthly = weather_pivot[['DATA_LECTURA', 'Temperatura mitjana diària', 'month', 'month_name']].dropna()
    temp_monthly = temp_monthly.sort_values('month')
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=temp_monthly, x='month_name', y='Temperatura mitjana diària', 
                order=temp_monthly.sort_values('month')['month_name'].unique(), color='coral')
    plt.title('Distribution of Average Daily Temperature by Month')
    plt.xlabel('Month')
    plt.ylabel('Temperature (°C)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()